# Feature Extraction Pipeline

This tutorial shows how to use the pipeline to extract features from a Physionet WFDB and CSV files on a disk.

### WFDB 

We extract features from the [ECGRDVQ](https://physionet.org/content/ecgrdvq/1.0.0/) database.  From the database description: "The ECGRDVQ database contains 5232 extracted 10-second standard 12 lead ECG segments".
The meta-data of the database (```ecgrdvq_metadata```), which is a dictionary consisting of the record names ```record_name``` and their coressponding public directories ```public_dir```, is avaialbe in ```cmda.data.wfdb```.
The pipeline includes importing the data, applying filters to the records and feature extraction from records. To import the data from Physionet WFDB we use the ```cmda.read_data.ReadWFDB```. We create the filtering object by ```cmda.filter.Filters``` and create the feature object by ```cmda.feature_extraction.Features```. A detailed tutorial of creating feature objects can be found here.

In [1]:
from cmda.data.wfdb import ecgrdvq_metadata
from cmda.read_data import ReadWFDB
from cmda.filter import Filters
from cmda.feature_extraction import Features
from cmda.pipeline import Pipeline

# Load the meta-data
metadata = ecgrdvq_metadata()

# get the record names and their corresponding public directories
# In this example we select the first 12 instances to have a fast results
record_names = metadata['record_name'][0:12]
public_dir = metadata['public_dir'][0:12]

# Build the importer object using ReadWFDB 
# Set the channels to ["II",'V1'] to import these channels exclusively. 
importer = ReadWFDB(record_names=record_names, public_dir=public_dir, channels=['II','V1'])

# Create the filter object and add a buuterworth low-pass filter
filters = Filters()
filters.add.butter_filter(cutoff=60, btype="lowpass")

# Create the feature object
features = Features()
features.add.mnf()
features.add.stdf()
features.add.band_sum(low=1, high=7)

In [2]:
# Build the pipeline
ecgrdvq_pipeline = Pipeline(importer=importer,features=features, filters=filters)

# Run the pipeline
# Set the dataframe_output to True, to get the extracted as a Pandas dataframe
res = ecgrdvq_pipeline.run(n_jobs=4, dataframe_output = True)

Running the pipeline on 12 instances...



100%|██████████| 12/12 [00:07<00:00,  1.53it/s]

finished!


In [3]:
# the extracted features
res

,II_mnf,II_stdf,II_band_sum,V1_mnf,V1_stdf,V1_band_sum
491af4aa-941a-4a89-b74c-b38d91cfc5e9,8.667959,6.152966,0.464750,7.560547,5.179558,0.503667
db4d09aa-f26c-4acb-92fd-6ac316918bc8,8.791343,6.136429,0.466031,7.498114,5.225458,0.512484
dd3caf18-354d-4c81-9ff5-6aed843cd84b,8.994343,6.254645,0.427771,7.572526,5.366888,0.492164
12133f6e-efcf-48cc-a184-2e4d7cc05a89,8.704130,6.257274,0.449003,7.631221,5.243020,0.497363
2c179592-3c18-47f2-930a-7f17ae4bc596,8.956668,6.766684,0.441981,7.723886,5.293620,0.484876
43457f03-eb84-49ef-a00a-8cbad7d5108d,9.574600,7.229321,0.375681,7.931221,8.430665,0.473490
00ed2097-cd14-4f03-ab33-853da5be5550,9.279138,6.419775,0.410092,7.540609,5.198409,0.493821
9d7c5729-9458-4f8f-bf32-532d6b5ee5a4,9.025684,6.204357,0.421404,7.546192,5.129932,0.491710
cd50da94-9c66-4b62-88a2-5879fbc17999,9.012871,6.206022,0.431222,7.681139,5.088155,0.488235
3289874c-4da9-4a88-b68f-517ebacf0862,8.821893,6.395041,0.429677,7.547925,5.285196,0.487406


## CSV files

In this example, we use a series of sample CSV files. The sample files contain three ECG channels ('II','V1','AVF'). To download the data, we use the ```cmda.data.get_sample.csv``` function. After downloading the data, we implement a CSV importer ```cmda.read_data.ReadCSV```. The rest of the procedure is similar to the previous example.

In [4]:
from cmda.data import get_csv_samples
from cmda.read_data import ReadCSV
from cmda.filter import Filters
from cmda.feature_extraction import Features
from cmda.pipeline import Pipeline

# Get the sample data
# Set the path on the disk for downloading the data
data_path = "/Users/pejman/Desktop/JRC/test"
get_csv_samples(data_path)


# Build the importer object using ReadCSV
# Give the data_path as files_list, which lead to import all the csv files in this directory
# Set the channels to None, to import all the channels
importer = ReadCSV(files_list = data_path, fs = 500, channels=None, index_col=0)

# In this example we apply no filters on the data

# Create the feature object
features = Features()
features.add.std()
features.add.mnf()
features.add.stdf()
features.add.band_sum(low=1, high=7)

finished!


In [5]:
# Build the pipeline
ecgrdvq_pipeline = Pipeline(importer=importer,features=features, filters=None)

# Run the pipeline
# Set the dataframe_output to True, to get the extracted as a Pandas dataframe
res = ecgrdvq_pipeline.run(n_jobs=4, dataframe_output = True)

Running the pipeline on 8 instances...



100%|██████████| 8/8 [00:00<00:00, 101.15it/s]

finished!


In [6]:
res

,II_std,II_mnf,II_stdf,II_band_sum,V1_std,V1_mnf,V1_stdf,V1_band_sum,AVF_std,AVF_mnf,AVF_stdf,AVF_band_sum
db4d09aa-f26c-4acb-92fd-6ac316918bc8.csv,0.213424,4.451252,3.550883,0.677450,0.107259,3.874733,3.984079,0.782436,0.179781,4.714349,3.791669,0.679989
dd3caf18-354d-4c81-9ff5-6aed843cd84b.csv,0.219672,4.552525,3.541276,0.706097,0.110938,3.913971,4.180290,0.787416,0.185088,4.796069,3.708513,0.703118
00ed2097-cd14-4f03-ab33-853da5be5550.csv,0.222349,4.696921,3.615635,0.681625,0.115206,3.881521,3.785739,0.791689,0.189489,4.925369,3.854928,0.678147
12133f6e-efcf-48cc-a184-2e4d7cc05a89.csv,0.222275,4.400237,3.494050,0.657455,0.097425,3.959626,4.289741,0.822707,0.188856,4.641844,3.680911,0.658898
2c179592-3c18-47f2-930a-7f17ae4bc596.csv,0.229621,4.528543,3.740004,0.664776,0.099633,4.009932,4.273336,0.811924,0.195285,4.747016,3.835435,0.663188
43457f03-eb84-49ef-a00a-8cbad7d5108d.csv,0.227854,4.849821,4.037641,0.701849,0.113948,4.106639,5.278077,0.774866,0.192569,5.036089,4.160120,0.695479
491af4aa-941a-4a89-b74c-b38d91cfc5e9.csv,0.215058,4.390563,3.606625,0.668551,0.108896,3.930762,4.531862,0.790218,0.181828,4.653420,3.895836,0.670621
9d7c5729-9458-4f8f-bf32-532d6b5ee5a4.csv,0.219417,4.568280,3.513118,0.667905,0.111342,3.888630,3.919959,0.784307,0.186903,4.799791,3.725385,0.666418
